In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (188 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
40 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acqu

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Creamos el Spark Context

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
import pandas as pd

from google.colab import drive
sqlContext = SQLContext(sc)

drive.mount("/content/drive")
DATA_PATH = "/content/drive/MyDrive/CienciaDeDatos/TP1/data/"

rdd_orders = (
    sqlContext.read.csv(DATA_PATH + 'orders.csv', header=True, inferSchema=True)
    .select("status", "shipping_address")
    .rdd
)

rdd_customers = (
    sqlContext.read.csv(DATA_PATH + 'customers.csv', header=True, inferSchema=True)
    .select("customer_id", "first_name", "postal_code")
    .rdd
)

TOP_CPS_SHOWED = 5
TOP_NAMES_SHOWED = 1

/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2) ¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado ‘Refunded’? ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

### Hipótesis Tomadas:
Se consideró que el código postal (CP) se extrae del shipping_address como el último espacio numérico (p.ej., en “3123 Alan Extension Port Andrea, MA 26926” el CP es 26926).

In [ ]:
def get_cp(address):
    try:
        cp = address.strip().split()[-1].lower().strip()
        if cp and cp.isdigit():
            return cp
        else:
            return None
    except:
        return None

rdd_refunded = rdd_orders.filter(
    lambda row: row["status"] is not None
    and row["shipping_address"] is not None
    and row["status"].strip().lower() == "refunded"
)

rdd_refunded_cps = rdd_refunded.map(lambda row: (
    get_cp(row["shipping_address"]),
    1
)).filter(lambda row: row[0] is not None)

cps_apparitions = rdd_refunded_cps.reduceByKey(lambda x, y: x + y)

top_cps = cps_apparitions.takeOrdered(TOP_CPS_SHOWED, key=lambda x: -x[1])

print(top_cps)

[('70696', 6), ('47612', 5), ('11954', 5), ('83755', 5), ('59883', 5)]


In [ ]:
rdd_top_cps = sc.parallelize(top_cps)

rdd_customers_by_cp = (
  rdd_customers
    .filter(lambda row: row["postal_code"] is not None and row["first_name"] is not None)
    .map(lambda row: (str(row["postal_code"]).lower().strip(),
                      row["first_name"].lower().strip()))
)

print("Clientes totales:", rdd_customers_by_cp.count())

customers_top_refunded_cps = rdd_customers_by_cp.join(rdd_top_cps)

print("Clientes en top CPs:", customers_top_refunded_cps.count())

name_apparitions = (
    customers_top_refunded_cps
    .map(lambda x: (x[1][0], 1))
    .reduceByKey(lambda a, b: a + b)
)

top_names = name_apparitions.takeOrdered(TOP_NAMES_SHOWED, key=lambda x: -x[1])

print(top_names)

Clientes totales: 399773
Clientes en top CPs: 14
[('kathy', 1), ('raymond', 1), ('diana', 1), ('dominique', 1), ('judith', 1), ('miranda', 1), ('jeffrey', 1), ('undefined', 1), ('shelly', 1), ('frank', 1), ('johnathan', 1), ('rhonda', 1), ('linda', 1), ('danielle', 1)]


# Conclusiones:

Esta vez no hubo nombres más frecuentes. Aunque estos resultados son consistentes con los obtenidos en el primer trabajo, al tratarse de un pipeline que paraleliza el procesamiento y como hay muchos CPs con 5 órdenes, los seleccionados esta vez fueron distintos. Por lo tanto no apareció un nombre para estas direcciones más de una vez. Solamente 14 clientes se encontraron en estas direcciones, todos de nombres distintos. Entre ellos: Kathy, Raymond y Diana.

Comentarios: La dispersión de CPs encontrada fue altísima. El CP más encontrado se repitió solo 6 veces. Debido a esto, el TOP de nombres fue muy pequeño.
Si se quisiera robustecer el análisis, se podría ampliar la ventana (Top 10–20 CP).
Saco la misma conclusión. Se extrae que, en el marco analizado, no se realizaron muchos reembolsos para un mismo lugar. Me atrevería a decir que debido a esto, parece que el sistema de compras/envío funciona bien y nadie abusa de los reembolsos, sino aparecería un nombre muchas veces.
